In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
# from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Activation, LeakyReLU, Dropout, BatchNormalization, MaxPooling2D, Lambda
from keras.models import Model
from sklearn.metrics import mean_squared_error as mse
from progressbar import ProgressBar
from image_process import ImageProcess
import const, cv2
from drive_data import DriveData
from config import Config

import matplotlib.pyplot as plt
import sys
sys.setrecursionlimit(10**7)
tf.compat.v1.disable_eager_execution()

In [ ]:
"""
## Dataset
"""
image_process = ImageProcess()
config = Config.neural_net
# data_path = "/mnt/exData/internal_simulation/test/2022-03-08-17-18-14/"
data_path = "/mnt/exData/internal_simulation/2022-12-08-14-25-28/"
if data_path[-1] == '/':
    data_path = data_path[:-1]

    loc_slash = data_path.rfind('/')
    if loc_slash != -1: # there is '/' in the data path
        model_name = data_path[loc_slash + 1:] # get folder name
        #model_name = model_name.strip('/')
    else:
        model_name = data_path
csv_path = data_path + '/' + model_name + const.DATA_EXT 
# data = DriveData(csv_path)
print(csv_path)
# data_path = data_path
# data.read()

csv_header = ['image_fname', 
                  'steering_angle', 'throttle', 'brake', 'linux_time', 
                  'vel', 'vel_x', 'vel_y', 'vel_z', 
                  'pos_x', 'pos_y', 'pos_z', 
                  'tar_image_fname', 'tar_steering_angle', 'tar_vel', 'tar_time']

df = pd.read_csv(csv_path, names=csv_header, index_col=False)
num_data = len(df)

bar = ProgressBar()
df_image_names = []
df_measurements = []
df_time_stamps= []
df_velocities= []
df_velocities_xyz= []
df_positions_xyz= []
df_tar_image_names= []
df_tar_steering_angle= []
df_tar_vel= []
df_tar_time= []
for i in bar(range(num_data)): # we don't have a title
    df_image_names.append(df.loc[i]['image_fname'])
    # if Config.data_collection['brake'] is True:
    df_measurements.append((float(df.loc[i]['steering_angle']),
                            float(df.loc[i]['throttle']), 
                            float(df.loc[i]['brake'])))
    df_time_stamps.append(float(df.loc[i]['linux_time']))
    df_velocities.append(float(df.loc[i]['vel']))
    df_velocities_xyz.append((float(df.loc[i]['vel_x']), 
                                float(df.loc[i]['vel_y']), 
                                float(df.loc[i]['vel_z'])))
    df_positions_xyz.append((float(df.loc[i]['pos_x']), 
                                float(df.loc[i]['pos_y']), 
                                float(df.loc[i]['pos_z'])))
    df_tar_image_names.append(df.loc[i]['tar_image_fname'])
    df_tar_steering_angle.append(float(df.loc[i]['tar_steering_angle']))
    df_tar_vel.append(float(df.loc[i]['tar_vel']))
    df_tar_time.append(float(df.loc[i]['tar_time']))

In [ ]:
samples = list(zip( df_image_names, df_velocities, df_measurements, 
                    df_tar_image_names, df_tar_steering_angle, df_tar_vel, df_tar_time))

In [ ]:
def prepare_vae_batch_samples(batch_samples):
    images = []
    steering_angles = []
    vels = []
    tar_images = []
    tar_steering_angles = []
    tar_vels = []
    tar_times = []
        
    for image_name, velocity, measurement, tar_image_name, tar_steering_angle, tar_vel, tar_time in batch_samples:
        # self.data.image_names, self.data.velocities, self.data.measurements, 
        # self.data.tar_image_names, self.data.tar_steering_angle, self.data.tar_vel, self.data.tar_time
        
        image_path = data_path + '/' + image_name
        # print(data_path, tar_image_name)
        tar_image_path = data_path + '/' + tar_image_name
        image = cv2.imread(image_path)
        tar_image = cv2.imread(tar_image_path)

        # if collected data is not cropped then crop here
        # otherwise do not crop.
        if Config.data_collection['crop'] is not True:
            image = image[Config.data_collection['image_crop_y1']:Config.data_collection['image_crop_y2'],
                        Config.data_collection['image_crop_x1']:Config.data_collection['image_crop_x2']]
            tar_image = tar_image[Config.data_collection['image_crop_y1']:Config.data_collection['image_crop_y2'],
                        Config.data_collection['image_crop_x1']:Config.data_collection['image_crop_x2']]
        image = cv2.resize(image, 
                            (config['input_image_width'],
                            config['input_image_height']))
        tar_image = cv2.resize(tar_image, 
                            (config['input_image_width'],
                            config['input_image_height']))
        image = image_process.process(image)
        tar_image = image_process.process(tar_image)

        vels.append(tar_vels)
        tar_vels.append(tar_vel)
        tar_times.append(tar_time)
        tar_steering_angles.append(tar_steering_angle)
        # if no brake data in collected data, brake values are dummy
        steering_angle, throttle, brake = measurement
        steering_angles.append(steering_angle)
        # cv2.imwrite('/home/kdh/oscar/oscar/e2e_fusion_data/test/aug/'+image_name, image)
        # if data == 'train':
        #     cv2.imwrite('/mnt/Data/oscar/train_data/'+image_name, image)
        # print(image.shape)
        images.append(image)
        tar_images.append(tar_image)
        # segimgs.append(segimg)


    return images, vels, steering_angles, tar_images, tar_steering_angles, tar_vels, tar_times


In [ ]:
"""
## Encoder
"""

"""
## Create a sampling layer
"""
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


encoder_inputs_img = tf.keras.Input(shape=(160, 160, 3))
encoder_inputs_str = tf.keras.Input(shape=(1))
encoder_inputs_vel = tf.keras.Input(shape=(1))
encoder_inputs_time = tf.keras.Input(shape=(1))
x = layers.Conv2D(24, (5, 5), padding='same', name='conv2d_1')(encoder_inputs_img)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2), name='pool2d_1')(x)

x = layers.Conv2D(36, (5, 5), padding='same', name='conv2d_2')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2), name='pool2d_2')(x)

x = layers.Conv2D(48, (5, 5), padding='same', name='conv2d_3')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)

x = layers.Conv2D(64, (3, 3), padding='same', name='conv2d_4')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)

x = layers.Conv2D(64, (3, 3), padding='same', name='conv2d_5')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('elu')(x)

latent = layers.Flatten()(x)
fc_s1  = layers.Dense(100)(encoder_inputs_str)
fc_s1  = layers.Activation('elu')(fc_s1)
fc_s2  = layers.Dense(50)(fc_s1)
fc_s2  = layers.Activation('elu')(fc_s2)
fc_v1  = layers.Dense(100)(encoder_inputs_vel)
fc_v1  = layers.Activation('elu')(fc_v1)
fc_v2  = layers.Dense(50)(fc_v1)
fc_v2  = layers.Activation('elu')(fc_v2)
fc_t1  = layers.Dense(100)(encoder_inputs_time)
fc_t1  = layers.Activation('elu')(fc_t1)
fc_t2  = layers.Dense(50)(fc_t1)
fc_t2  = layers.Activation('elu')(fc_t2)
conc_1 = layers.concatenate([latent, fc_s2, fc_v2, fc_t2])
fc_1   = layers.Dense(100)(conc_1)
x   = layers.Activation('elu')(fc_1)

z_mean = layers.Dense(50, name="z_mean")(x)
z_log_var = layers.Dense(50, name="z_log_var")(x)
encoder_output = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model([ encoder_inputs_img, encoder_inputs_str,
                        encoder_inputs_vel, encoder_inputs_time], 
                        [z_mean, z_log_var, encoder_output], 
                        name="encoder")
encoder.summary()

In [ ]:
"""
## Decoder
"""

latent_inputs = tf.keras.Input(shape=(50,))
x = layers.Dense(40 * 40 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((40, 40, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
"""
## VAE
"""
model_input = [ encoder_inputs_img, encoder_inputs_str,
                        encoder_inputs_vel, encoder_inputs_time]
model_output = decoder(encoder_output)

VarAE=Model(model_input, model_output)
# VarAE.run_eagerly = True


In [ ]:
"""
VAE loss
"""
optimizer=tf.keras.optimizers.Adam(lr=0.0005)
r_loss_factor=1000   # This is a Hyperparameter

def vae_r_loss(y_true, y_pred):    ## MSE
    r_loss = tf.keras.backend.mean(tf.keras.backend.square(y_true-y_pred), axis=[1,2,3])
    return r_loss_factor * r_loss

def vae_kl_loss(y_true, y_pred):   ## KL-Divergence
    kl_loss=( -0.5 * tf.keras.backend.sum(1+z_log_var 
                - tf.keras.backend.square(z_mean) 
                - tf.keras.backend.exp(z_log_var), axis=1)
    )
    return kl_loss

def vae_loss(y_true, y_pred): 
    r_loss=vae_r_loss(y_true, y_pred) #Loss of Decoder
    kl_loss = vae_kl_loss(y_true, y_pred) #Loss of Encoder
    return r_loss + kl_loss #Sum of these two


VarAE.compile(optimizer=optimizer,
                loss= vae_loss, 
                metrics=[vae_r_loss, vae_kl_loss])
VarAE.summary()

In [ ]:
"""
Data generator
"""
from sklearn.model_selection import train_test_split

def _generator(samples, batch_size=config['batch_size']):
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates
        for offset in range(0, (num_samples//batch_size)*batch_size, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images, vels, steering_angles, tar_images, tar_steering_angles, tar_vels, tar_times = prepare_vae_batch_samples(batch_samples)

            X_img = np.array(images).astype("float64")/255.0
            X_tvel = np.array(tar_vels)
            X_tstr = np.array(tar_steering_angles)
            X_ttime = np.array(tar_times)

            y_train = np.array(tar_images).astype("float64")/255.0
            # if config['num_outputs'] == 1:
            # y_train = np.array(segimg)
            # print(y_train.max())
            # y_train = np.repeat(y_train[..., np.newaxis], 1, -1)/y_train.max()
            X_train = [X_img, X_tstr, X_tvel, X_ttime]
            # print(X_train_vel.shape)
            # print(y_train.shape)
            yield X_train, y_train

train_data, valid_data = train_test_split(samples, test_size=config['validation_rate'])

train_generator = _generator(train_data)
valid_generator = _generator(valid_data)    


In [ ]:

"""
## Train the VAE
"""
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.python.framework.ops import disable_eager_execution

        # checkpoint
callbacks = []
#weight_filename = self.data_path + '_' + Config.config_yaml_name \
#    + '_N' + str(config['network_type']) + '_ckpt'
model_ckpt_name = "vae_kdh"
checkpoint = ModelCheckpoint(model_ckpt_name +'_ckpt.{epoch:02d}-{val_loss:.3f}.h5',
                                monitor='val_loss', 
                                verbose=1, save_best_only=True, mode='min')
callbacks.append(checkpoint)

# early stopping
patience = config['early_stopping_patience']
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, 
                            verbose=1, mode='min')
callbacks.append(earlystop)

# validation_steps = len(valid_data)//config['batch_size']
# print(config['batch_size'])
train_hist = VarAE.fit( train_generator,  
                        steps_per_epoch=len(train_data)//config['batch_size'], 
                        epochs=config['num_epochs'], 
                        validation_data=valid_generator,
                        validation_steps=len(valid_data)//config['batch_size'],
                        shuffle=True,
                        verbose=1, 
                        callbacks=callbacks, 
                        use_multiprocessing=True,
                        workers=48
                        )


# train_hist = VarAE.fit( train_generator,  
#                         batch_size=config['batch_size'],
#                         steps_per_epoch=len(train_data)//config['batch_size'], 
#                         epochs=config['num_epochs'], 
#                         validation_data=valid_generator,
#                         validation_steps=len(valid_data)//config['batch_size'],
#                         shuffle=True,
#                         verbose=1, 
#                         callbacks=callbacks, 
#                         use_multiprocessing=True,
#                         workers=48
#                         )
    